In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
from xgboost import XGBClassifier

In [5]:
df = pd.read_csv('../../datasets/homeCredit/BDSE12_03G_HomeCredit_V2.csv')
df.replace([np.inf, -np.inf], np.nan,inplace=True)

def process_dataframe(input_df, encoder_dict=None):
    """ Process a dataframe into a form useable by LightGBM """

    # Label encode categoricals
    categorical_feats = input_df.columns[input_df.dtypes == 'object']
    categorical_feats = categorical_feats
    encoder_dict = {}
    for feat in categorical_feats:
        encoder = LabelEncoder()
        input_df[feat] = encoder.fit_transform(input_df[feat].fillna('NULL'))
        encoder_dict[feat] = encoder

    return input_df, categorical_feats.tolist(), encoder_dict

df, categorical_feats, encoder_dict = process_dataframe(input_df=df)

In [6]:
def lgbm_evaluate(**params):
    warnings.simplefilter('ignore')
    
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
        
    clf = LGBMClassifier(**params, 
                         n_estimators = 2000,
                         nthread = 5, 
                         boosting_type='rf', 
                         objective='binary')

    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]

    folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=1001)
        
    test_pred_proba = np.zeros(train_df.shape[0])
    
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = False, early_stopping_rounds = 100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(train_df['TARGET'], test_pred_proba)

In [4]:
def xgb_evaluate(**params):
    warnings.simplefilter('ignore')
    
    params['max_depth'] = int(params['max_depth'])
        
    clf = XGBClassifier(**params, 
                        n_estimators = 2000, 
                        nthread = 5, 
                        objective= 'binary:logistic')

    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]

    folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=1001)
        
    test_pred_proba = np.zeros(train_df.shape[0])
    
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = False, early_stopping_rounds = 100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration)[:, 1]
        
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(train_df['TARGET'], test_pred_proba)

In [ ]:
init_time = time.time()
params = {'learning_rate': (.01, .03), 
          'subsample': (.0, 1.0), 
          'max_depth': (4, 9), 
          'reg_alpha': (.0, 1.0), 
          'reg_lambda': (.0, 1.0), 
          'scale_pos_weight': (.0, 5.0),
          'colsample_bytree': (.0, 1.0)}
bo = BayesianOptimization(xgb_evaluate, params)
bo.maximize(init_points = 5, n_iter = 5)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

In [8]:
init_time = time.time()
params = {'learning_rate': (.01, .03), 
          'num_leaves': (20, 100), 
          'subsample': (0.8, 1), 
          'max_depth': (6, 9), 
          'reg_alpha': (.00, 1.0), 
          'reg_lambda': (.00, 1.0), 
          'min_split_gain': (.01, .03),
          'min_child_weight': (20, 70),
          'subsample_freq':(0, 10)}
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 5, n_iter = 5)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

|   iter    |  target   | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------


LightGBMError: Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at d:\a\1\s\python-package\compile\src\boosting\rf.hpp, line 29 .


In [15]:
init_time = time.time()
params = {'learning_rate': (.01, .03), 
          'num_leaves': (20, 100), 
          'subsample': (0.8, 1), 
          'max_depth': (6, 9), 
          'reg_alpha': (.00, 1.0), 
          'reg_lambda': (.00, 1.0), 
          'min_split_gain': (.01, .03),
          'min_child_weight': (20, 70)}
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 5, n_iter = 5)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

|   iter    |  target   | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7935   |  0.01112  |  7.106    |  44.6     |  0.01451  |  91.72    |  0.09609  |  0.3668   |  0.9818   |
|  2        |  0.7932   |  0.02896  |  6.387    |  57.54    |  0.02782  |  23.84    |  0.05095  |  0.6451   |  0.8986   |
|  3        |  0.793    |  0.02692  |  7.423    |  62.36    |  0.02013  |  92.88    |  0.6868   |  0.8196   |  0.9318   |
|  4        |  0.7934   |  0.01211  |  6.622    |  68.66    |  0.01294  |  54.74    |  0.108    |  0.3523   |  0.8212   |
|  5        |  0.7938   |  0.01929  |  8.672    |  67.28    |  0.0162   |  33.82    |  0.5987   |  0.6734   |  0.9283   |
|  6        |  0.7929   |  0.01324  |  8.177    |  20.21    |  0.01961  |  20.01    |  0.4762   |  0.7955   |  0.8782   |
|  7        |  0.7917   

In [16]:
bo.res[5]

{'target': 0.792940652843537,
 'params': {'learning_rate': 0.013237147439636492,
  'max_depth': 8.176972909272658,
  'min_child_weight': 20.210808853599698,
  'min_split_gain': 0.019613448600170856,
  'num_leaves': 20.01281707147962,
  'reg_alpha': 0.4762374177443556,
  'reg_lambda': 0.795459509951648,
  'subsample': 0.8782478787186309}}

In [17]:
init_time = time.time()
params = {'learning_rate': (.0, .1), 
          'num_leaves': (20, 100), 
          'subsample': (.0, 1.0), 
          'max_depth': (6, 9), 
          'reg_alpha': (.00, 1.0), 
          'reg_lambda': (.00, 1.0), 
          'min_split_gain': (.0, .1),
          'min_child_weight': (20, 70)}
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 5, n_iter = 10)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

|   iter    |  target   | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.789    |  0.06537  |  8.788    |  22.77    |  0.08022  |  60.89    |  0.9679   |  0.2213   |  0.4622   |
|  2        |  0.7921   |  0.0361   |  8.687    |  41.12    |  0.08093  |  61.53    |  0.8299   |  0.4435   |  0.6061   |
|  3        |  0.7937   |  0.0138   |  8.885    |  57.73    |  0.07277  |  81.73    |  0.6619   |  0.3204   |  0.5381   |
|  4        |  0.7909   |  0.07356  |  7.91     |  63.69    |  0.04934  |  44.67    |  0.05158  |  0.6932   |  0.08962  |
|  5        |  0.7905   |  0.07356  |  6.978    |  62.94    |  0.08465  |  58.14    |  0.5698   |  0.2101   |  0.3065   |
|  6        |  0.7911   |  0.04674  |  8.316    |  69.69    |  0.06259  |  99.95    |  0.1455   |  0.6733   |  0.2921   |
|  7        |  0.792    

In [19]:
bo.res[2]

{'target': 0.7936603978356124,
 'params': {'learning_rate': 0.013803211212539657,
  'max_depth': 8.884921565475697,
  'min_child_weight': 57.725650927715265,
  'min_split_gain': 0.07276947619457204,
  'num_leaves': 81.72909839300354,
  'reg_alpha': 0.6618734111073816,
  'reg_lambda': 0.320433363007782,
  'subsample': 0.5381263969882377}}